In [2]:
import sys
import os

# add project root to Python path
sys.path.append(os.path.abspath(".."))


In [4]:
from src.load_to_db import load_csv_to_sqlite


In [7]:
from src.load_to_db import load_csv_to_sqlite

load_csv_to_sqlite("../data/games_2024.csv", "games")


Loaded ../data/games_2024.csv into table 'games'


In [8]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../database/nba.db")


In [9]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)


,name
0,games


In [10]:
pd.read_sql("SELECT * FROM games LIMIT 5;", conn)


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22023,1610612743,DEN,Denver Nuggets,22300061,2023-10-24,DEN vs. LAL,W,240,48,...,33,42,29,9,6,12,15,119,12,1
1,22023,1610612744,GSW,Golden State Warriors,22300062,2023-10-24,GSW vs. PHX,L,240,36,...,31,49,19,11,6,11,23,104,-4,1
2,22023,1610612747,LAL,Los Angeles Lakers,22300061,2023-10-24,LAL @ DEN,L,240,41,...,31,44,23,5,4,12,18,107,-12,1
3,22023,1610612756,PHX,Phoenix Suns,22300062,2023-10-24,PHX @ GSW,W,240,42,...,43,60,23,5,7,19,22,108,4,1
4,22023,1610612737,ATL,Atlanta Hawks,22300063,2023-10-25,ATL @ CHA,L,240,39,...,30,42,24,12,1,12,19,110,-6,1


In [13]:
pd.read_sql("PRAGMA table_info(games);", conn)


,cid,name,type,notnull,dflt_value,pk
0,0,SEASON_ID,INTEGER,0,None,0
1,1,TEAM_ID,INTEGER,0,None,0
2,2,TEAM_ABBREVIATION,TEXT,0,None,0
3,3,TEAM_NAME,TEXT,0,None,0
4,4,GAME_ID,INTEGER,0,None,0
5,5,GAME_DATE,TEXT,0,None,0
6,6,MATCHUP,TEXT,0,None,0
7,7,WL,TEXT,0,None,0
8,8,MIN,INTEGER,0,None,0
9,9,FGM,INTEGER,0,None,0


In [14]:
df = pd.read_sql("SELECT * FROM games", conn)


In [15]:
df['is_home'] = df['MATCHUP'].str.contains("vs")
df['is_away'] = df['MATCHUP'].str.contains("@")


In [16]:
df['opponent'] = df['MATCHUP'].str.split().str[-1]


In [17]:
clean = df[['GAME_ID', 'TEAM_ABBREVIATION', 'opponent', 'is_home', 'is_away', 
            'PTS', 'REB', 'AST', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'WL']]


In [18]:
pd.read_sql("""
SELECT TEAM_ABBREVIATION AS team, PTS
FROM games
ORDER BY PTS DESC
LIMIT 10;
""", conn)


,team,PTS
0,IND,157
1,IND,157
2,BOS,155
3,UTA,154
4,NOP,153
5,IND,152
6,ATL,152
7,LAC,151
8,IND,150
9,LAL,150


In [19]:
suns = pd.read_sql("""
SELECT *
FROM games
WHERE TEAM_ABBREVIATION = 'PHX';
""", conn)


In [20]:
df_self = df[['GAME_ID', 'TEAM_ABBREVIATION', 'PTS']]
df_opp = df[['GAME_ID', 'opponent', 'PTS']].rename(columns={'opponent':'TEAM_ABBREVIATION', 'PTS':'OPP_PTS'})

merged = pd.merge(df_self, df_opp, on=['GAME_ID', 'TEAM_ABBREVIATION'])


In [12]:
pd.read_sql("""
SELECT *
FROM (
    SELECT Home AS team, HomePts AS points FROM games
    UNION ALL
    SELECT Visitor AS team, VisitorPts AS points FROM games
)
ORDER BY points DESC
LIMIT 10;
""", conn)


DatabaseError: Execution failed on sql '
SELECT *
FROM (
    SELECT Home AS team, HomePts AS points FROM games
    UNION ALL
    SELECT Visitor AS team, VisitorPts AS points FROM games
)
ORDER BY points DESC
LIMIT 10;
': no such column: Visitor